In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv")
df_1=pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv")

*Exploratory Data Ananlysis*

In [ ]:
df.head()

In [ ]:
print(df['date'].head(10))

Prints a statistical summary of numerical columns and lists the categorical and numerical features.

In [ ]:
# Statistical summary
print(df.describe())

# Checking feature types
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns

print("\nCategorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

*Plot a histogram of the 'purchaseValue' column to visualize its distribution and check for outliers.*

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['purchaseValue'], bins=50, kde=True)
plt.title("Distribution of Purchase Value")
plt.xlabel("Purchase Value")
plt.show()

***Creating scatter plots to visualize the relationship between the specified numerical features and the target variable 'purchaseValue'.**

In [ ]:
numerical_features_to_plot = ['totalHits', 'pageViews', 'sessionNumber']

for col in numerical_features_to_plot:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x=col, y='purchaseValue', alpha=0.5)
    plt.title(f"Relationship between {col} and Purchase Value")
    plt.xlabel(col)
    plt.ylabel("Purchase Value")
    plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
missing_percent = df.isnull().mean() * 100

cols_to_drop = missing_percent[missing_percent > 50].index

print("Columns to Drop:", list(cols_to_drop))

# Drop these columns
df = df.drop(columns=cols_to_drop)
df_1= df_1.drop(columns=cols_to_drop)

print("New Train Shape:", df.shape)
print("New Test Shape:", df_1.shape)

In [ ]:
constant_cols = [col for col in df.columns if df[col].nunique() == 1]

print("Columns with only one unique value:\n", constant_cols)

df = df.drop(columns=constant_cols)

print("\nShape after dropping:", df.shape)

In [ ]:
constant_cols = [col for col in df_1.columns if df_1[col].nunique() == 1]

print("Columns with only one unique value:\n", constant_cols)

df_1 = df_1.drop(columns=constant_cols)

print("\nShape after dropping:", df_1.shape)

In [ ]:
cardinality = df.nunique()

print("Cardinality of each column:\n", cardinality)

high_cardinality = cardinality[cardinality > 100]
print("High-Cardinality Columns:\n", high_cardinality)

In [ ]:
cardinality = df_1.nunique()

print("Cardinality of each column:\n", cardinality)

high_cardinality = cardinality[cardinality > 100]
print("High-Cardinality Columns:\n", high_cardinality)

In [ ]:
cols_to_drop=[
"geoNetwork.region",
"trafficSource",
"sessionId",
"locationCountry",
"geoNetwork.city",
"geoNetwork.metro"]
df=df.drop(columns=cols_to_drop)
df_1=df_1.drop(columns=cols_to_drop)

In [ ]:
df.shape

In [ ]:
df_1.shape

In [ ]:
target = 'purchaseValue'

num_cols = df.select_dtypes(include=['int64', 'float64']).columns

correlation = df[num_cols].corr()[target].sort_values(ascending=False)

print("Correlation of Numerical Features with Target:\n")
print(correlation)


In [ ]:
df['hits_per_page'] = df['totalHits'] / (df['pageViews'] + 1)
df_1['hits_per_page'] = df_1['totalHits'] / (df_1['pageViews'] + 1)

In [ ]:
df['engagement_per_session'] = (df['totalHits'] + df['pageViews']) / (df['sessionNumber'] + 1)
df_1['engagement_per_session'] = (df_1['totalHits'] + df_1['pageViews']) / (df_1['sessionNumber'] + 1)

In [ ]:
train=df
test=df_1

In [ ]:
X = train.drop(columns=["purchaseValue"])
y = train["purchaseValue"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
!pip install category_encoders

In [ ]:
import category_encoders as ce

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols = X_train.select_dtypes(include=["int64","float64"]).columns
cat_cols = X_train.select_dtypes(include=["object"]).columns

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler",  StandardScaler())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("target_enc", ce.TargetEncoder())
])

preprocessor = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])


In [ ]:
X_train_processed = preprocessor.fit_transform(X_train,y_train)
X_test_processed = preprocessor.transform(X_test)
test_processed = preprocessor.transform(test)


In [ ]:
from sklearn.linear_model import LinearRegression

linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_processed, y_train)

print("Linear Regression model trained successfully!")

In [ ]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(random_state=42)
lgbm_model.fit(X_train_processed, y_train)

print("LightGBM model trained successfully!")

In [ ]:
from xgboost import XGBRegressor

In [ ]:
pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", XGBRegressor(random_state=42))
])

In [ ]:
xgb_model = XGBRegressor(
    colsample_bytree=0.600,
    learning_rate=0.0450,
    max_depth=10,
    n_estimators=1300,
    reg_alpha=2.0,
    reg_lambda=3.0,
    subsample=0.9500,
    tree_method="hist",
    verbosity=0,
    random_state=42,
    n_jobs=2
)


In [ ]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

In [ ]:
X_full_processed = preprocessor.fit_transform(X_full, y_full)

xgb_model.fit(X_full_processed, y_full)

print("Final Model Trained Successfully!")

In [ ]:
print("🔄 Predicting on Test Dataset...")

test_processed = preprocessor.transform(test)

test_predictions = xgb_model.predict(test_processed)

print("Prediction Complete!")

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

linear_regression_predictions = linear_regression_model.predict(X_test_processed)
lgbm_predictions = lgbm_model.predict(X_test_processed)
xgb_predictions = xgb_model.predict(X_test_processed)

lr_r2 = r2_score(y_test, linear_regression_predictions)
lr_mse = mean_squared_error(y_test, linear_regression_predictions)
print(f"Linear Regression R-squared: {lr_r2:.4f}")
print(f"Linear Regression Mean Squared Error: {lr_mse:.4f}")

lgbm_r2 = r2_score(y_test, lgbm_predictions)
lgbm_mse = mean_squared_error(y_test, lgbm_predictions)
print(f"LightGBM R-squared: {lgbm_r2:.4f}")
print(f"LightGBM Mean Squared Error: {lgbm_mse:.4f}")

xgb_r2 = r2_score(y_test, xgb_predictions)
xgb_mse = mean_squared_error(y_test, xgb_predictions)
print(f"XGBoost R-squared: {xgb_r2:.4f}")
print(f"XGBoost Mean Squared Error: {xgb_mse:.4f}")

Insights we gained:
The dataset was preprocessed by dropping columns with more than 50% missing values, constant values, and high cardinality. New features, such as hits_per_page and engagement_per_session, were introduced.
Three regression models (Linear Regression, LightGBM, and XGBoost) were trained on the preprocessed data.
The models were evaluated using R-squared and Mean Squared Error on a test set.
LightGBM achieved an R-squared of 0.1705 and MSE of 39,207,798,099,944,456.0000.
Linear Regression achieved an R-squared of 0.1367 and MSE of 40,808,877,192,487,896.0000.
XGBoost achieved an R-squared of 0.1137 and MSE of 41,895,806,342,262,776.0000.
Based on the evaluation metrics, LightGBM performed slightly better than Linear Regression and XGBoost on this dataset.

In [ ]:
submission = pd.DataFrame({
    "id": range(len(test_predictions)),
    "purchaseValue": test_predictions
})

submission.to_csv('submission.csv',index=False)
print("SUbmission file created successfully")